app: nombre de la app (inDrive, DiDi, Uber, Cabify, TaxisLibres)

ciudad: ciudad colombiana analizada

maw_est_low: estimación baja de usuarios activos

maw_est_high: estimación alta de usuarios activos

drivers_est_low: estimación baja de conductores activos

drivers_est_high: estimación alta de conductores activos

In [9]:
# estimacion_apps_ciudad.py
# Requisitos: pandas, numpy, matplotlib (pip install pandas numpy matplotlib)
# Uso: python estimacion_apps_ciudad.py
import pandas as pd
import numpy as np

# --- Datos base (puedes actualizar estos valores con datos pagados)
# Apps que estamos analizando
apps = [
    {"app":"inDrive", "maw_nacional_est_range": (2500000, 3100000)},   # MAU estimado nacional (ejemplo)
    {"app":"DiDi",   "maw_nacional_est_range": (1200000, 1600000)},
    {"app":"Uber",   "maw_nacional_est_range": (900000, 1300000)},
    {"app":"Cabify", "maw_nacional_est_range": (200000, 500000)},
    {"app":"TaxisLibres", "maw_nacional_est_range": (100000, 300000)},
]

# Ciudades y población (datos de ejemplo; reemplaza por cifras oficiales)
ciudades = [
    {"ciudad":"Bogotá", "poblacion": 8100000, "flota_taxis": 58247},
    {"ciudad":"Medellín", "poblacion": 2550000, "flota_taxis": 19000},
    {"ciudad":"Cali", "poblacion": 1200000, "flota_taxis": 13000},
    {"ciudad":"Barranquilla", "poblacion": 1200000, "flota_taxis": 12500},
    {"ciudad":"Bucaramanga", "poblacion": 580000, "flota_taxis": 6000},
]

df_ciudades = pd.DataFrame(ciudades)
total_pop = df_ciudades['poblacion'].sum()
df_ciudades['pop_share'] = df_ciudades['poblacion'] / total_pop

# Modelo: repartir MAU nacional por población urbana como proxy,
# ajustar para que ciudades grandes tengan +20% por penetración (hipótesis)
def city_penalty_adjust(share, poblacion):
    # ejemplo simple: boost para ciudades > 2M
    if poblacion > 2000000:
        return share * 1.2
    return share

df_ciudades['adj_share'] = df_ciudades.apply(lambda r: city_penalty_adjust(r['pop_share'], r['poblacion']), axis=1)
# normalizar
df_ciudades['adj_share'] = df_ciudades['adj_share'] / df_ciudades['adj_share'].sum()

# Ejecutar estimaciones
rows = []
for app in apps:
    low, high = app['maw_nacional_est_range']
    for _, c in df_ciudades.iterrows():
        est_low = low * c['adj_share']
        est_high = high * c['adj_share']
        # Estimar conductores:
        # - si app es taxi (TaxisLibres), usar flota_taxis * tasa_uso_app (ej: 0.6-0.9)
        # - si app es ride-hailing (Uber/DiDi/inDrive), estimar conductores como % de taxis+parque privado
        if app['app'] == 'TaxisLibres':
            driver_low = c['flota_taxis'] * 0.4
            driver_high = c['flota_taxis'] * 0.7
        else:
            # hipótesis: entre 5% y 25% de la flota taxi+parque privado usan app como conductor activo
            driver_low = c['flota_taxis'] * 0.05
            driver_high = c['flota_taxis'] * 0.25
        rows.append({
            "app": app['app'],
            "ciudad": c['ciudad'],
            "maw_est_low": int(est_low),
            "maw_est_high": int(est_high),
            "drivers_est_low": int(driver_low),
            "drivers_est_high": int(driver_high)
        })

df_out = pd.DataFrame(rows)
df_out.to_csv("estimacion_apps_ciudad.csv", index=False)
print("Estimaciones creadas -> estimacion_apps_ciudad.csv")
print(df_out)

# Visual simple (opcional)
import matplotlib.pyplot as plt
for app in df_out['app'].unique():
    d = df_out[df_out['app']==app]
    plt.figure(figsize=(8,3))
    plt.bar(d['ciudad'], d['maw_est_high'])
    plt.title(f"MAU estimado (upper bound) - {app}")
    plt.ylabel("Usuarios activos (estimado)")
    plt.tight_layout()
    plt.show()


Estimaciones creadas -> estimacion_apps_ciudad.csv
            app        ciudad  maw_est_low  maw_est_high  drivers_est_low  \
0       inDrive        Bogotá      1541878       1911928             2912   
1       inDrive      Medellín       485406        601903              950   
2       inDrive          Cali       190355        236040              650   
3       inDrive  Barranquilla       190355        236040              625   
4       inDrive   Bucaramanga        92005        114086              300   
5          DiDi        Bogotá       740101        986802             2912   
6          DiDi      Medellín       232994        310659              950   
7          DiDi          Cali        91370        121827              650   
8          DiDi  Barranquilla        91370        121827              625   
9          DiDi   Bucaramanga        44162         58883              300   
10         Uber        Bogotá       555076        801776             2912   
11         Uber      Mede


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\crist\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\crist\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\crist\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\Users\crist\anaconda3\Lib\site-pack

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

In [44]:
df_out

,app,ciudad,maw_est_low,maw_est_high,drivers_est_low,drivers_est_high
0,inDrive,Bogotá,1541878,1911928,2912,14561
1,inDrive,Medellín,485406,601903,950,4750
2,inDrive,Cali,190355,236040,650,3250
3,inDrive,Barranquilla,190355,236040,625,3125
4,inDrive,Bucaramanga,92005,114086,300,1500
5,DiDi,Bogotá,740101,986802,2912,14561
6,DiDi,Medellín,232994,310659,950,4750
7,DiDi,Cali,91370,121827,650,3250
8,DiDi,Barranquilla,91370,121827,625,3125
9,DiDi,Bucaramanga,44162,58883,300,1500
